# 命题逻辑

设计一种逻辑语言的目的是使推理形式更明确，因此它可以在自然语言中确定一组句子是否是一致的。作为这种方法的一部分，我们需要开发一个句子 φ 的逻辑表示，它能形式化地捕捉 φ 为真的条件。我们先从一个简单的例子开始：

    [Klaus chased Evi] and [Evi ran away].

让我们分别用 φ 和 ψ 替换其中的两个子句，并用 & 代替对应的英文单词 and 的逻辑操作，那么该句子就可以用 φ & ψ 的**逻辑形式**来表示。

**命题逻辑**使我们能只表示语言结构中句子连接词对应的部分，除了刚才看到的 and，还有 not、or、if then 等，它们被称作**布尔运算符**。命题逻辑的基本表达式是**命题符号**，通常写作 P、Q、R 等。接下来，我们重点研究 NLTK 中的逻辑表示方式：

In [1]:
import nltk

print(nltk.boolean_ops())

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->
None


从**命题符号**和**布尔运算符**，我们可以建立命题逻辑的**规范公式**（或简称公式）的无限集合。首先，每个命题字母是一个公式，然后，如果 φ 是一个公式，那么 -φ 也是一个公式。如果 φ 和 ψ 是公式，那么 (φ & ψ)、(φ | ψ)、(φ -> ψ) 和 (φ <-> ψ) 也都是公式。

下表展示了包含这些运算符的公式为真的条件，这里 iff 是 if and only if（当且仅当）的缩写：

| 布尔运算符               | 真值条件                                                                   |
|--------------------------|----------------------------------------------------------------------------|
| 非（否定的情况）         | 在 s 中 -φ 为真              iff       在 s 中 φ 为假                      |
| 与（and）                | 在 s 中 (φ & ψ) 为真         iff       在 s 中 φ 为真并且 ψ 为真           |
| 或（or）                 | 在 s 中 (φ &#124; ψ) 为真         iff       在 s 中 φ 为真或者 ψ 为真           |
| 蕴含（if ..., then ...） | 在 s 中 (φ -> ψ) 为真        iff       在 s 中 φ 为假或者 ψ 为真           |
| 等价（if and only if）   | 在 s 中 (φ <-> ψ) 为真       iff       在 s 中 φ 和 ψ 同时为真或者同时为假 |

这些规则通常比较简单，除了**蕴含**的真值条件违反了很多我们通常关于英语中条件句的直觉。形式 (P -> Q) 的公式为假只有当 P 为真并且 Q 为假时才成立。如果 P 为假（例如 P 对应 The moon is made of gree cheese）而 Q 为真（例如 Q 对应 Two plus two equals four），那么 P -> Q 的结果是真。

NLTK 的 [Expression](https://www.nltk.org/_modules/nltk/sem/logic.html#Expression) 对象可以将逻辑表达式分析成 Expression 的各种子类：

In [2]:
read_expr = nltk.sem.Expression.fromstring
print(type(read_expr('- (P & Q)')))
print(type(read_expr('P & Q')))
print(type(read_expr('P | (R -> Q)')))
print(type(read_expr('P <-> -- P')))

<class 'nltk.sem.logic.NegatedExpression'>
<class 'nltk.sem.logic.AndExpression'>
<class 'nltk.sem.logic.OrExpression'>
<class 'nltk.sem.logic.IffExpression'>


从计算的角度来看，逻辑给了我们进行推理的一个重要工具。假设你要表达：Freedonia is not to the north of Sylvania，而你给出的理由是 Sylvania is to the north of Freedonia，那么在这种情况下，你已经给出了一个**论证**。句子 Sylvania is to the north of Freedonia 是论证的**假设**，而 Freedonia is not to the north of Sylvania 是论证的**结论**。从假设一步一步推导到结论，被称为**推理**。一个论证，如果不是它的所有前提都是真而结论是假这种情况，那么它就是**有效**的！

更进一步地说，上述论证是否有效取决于短语 to the north of 的含义，如果它是对称的，那么该论证就无效，如果它是非对称的，那么该论证就有效。实际上，这个短语是一个非对称关系：

    if x is to the north of y then y is not to the north of x.

不幸的是，在命题逻辑中我们不能表达这样的规则，因为我们能用的最小元素就是原子命题（x is to the north of y），而不能“向里看”来讨论个体 x 和 y 之间的关系。在这种情况下，我们可以做的最好的是捕捉不对称的一个特定案例。让我们用命题符号 SnF 表示 Sylvania is to the north of Freedonia，用 FnS 表示 Freedonia is to the north of Sylvania。要说 Freedonia is not to the north of Sylvania，我们写成 -FnS。分别替代 x 和 y 为 Sylvania 和 Freedonia，于是我们又可以得到一个蕴含:

    SnF -> -FnS
    
我们用 [A1, ..., An] / C 代表一个从假设 [A1, ..., An] 得出结论 C 的论证。那么前面的例子就可以表示为：

    [SnF, SnF -> -FnS] / -FnS
    
这是一个有效的论证，SnF 为真，SnF -> -FnS 也为真，因此 -FnS 也一定为真。我们可以在 NLTK 中通过一个第三方定理证明器 Prover9 来进行逻辑证明：

In [3]:
read_expr = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

True

这里还有另一种方式可以看到结论如何得出：x -> y 在语义上等价于 -x | y，因此 SnF -> -FnS 为真也就是说 -SnF | -FnS 为真，又 SnF 为真，即 -SnF 为假，那么 -FnS 一定为真了。

回想一下，一个命题逻辑的模型需要为每个可能的公式分配值 True 或 Fasle。我们一步步地来实现这个模型：首先，为每个命题符号分配一个值，然后确定布尔运算符的含义和运用它们到这些公式的效果，来计算复杂的公式值。估值（[Valuation](http://www.nltk.org/_modules/nltk/sem/evaluate.html#Valuation)）是从逻辑的基本符号映射到它们的值，例如：

In [4]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])
print(val['P'])

True


估值对象基本上只是一个字典，映射逻辑符号为适当的值。接下来我们声明参数 dom 和 g，并用 val 初始化模型 m。每一个模型都有一个 evaluate() 方法，可以确定逻辑表达式的值：

In [5]:
dom = set()
g = nltk.Assignment(dom)
m = nltk.Model(dom, val)
print(m.evaluate('(P & Q)', g))
print(m.evaluate('-(P & Q)', g))
print(m.evaluate('(P & R)', g))
print(m.evaluate('(P | R)', g))

True
False
False
True


到目前为止，我们已经将我们的英文句子翻译成命题逻辑，其中我们只限于用字母 P 和 Q 表示原子句子，不能深入其内部结构。如果我们想要形式化前面如 to the north of 这样的论证，就必须要能看到里面基本的句子。因此我们将超越命题逻辑到一个更有表现力的东西，也就是下一节要讲的**一阶逻辑**。